In [311]:
import numpy as np
from riverswim import RiverSwim
import copy 

np.set_printoptions(precision=3)

In [325]:
def bellman_backup(state, action, R, T, gamma, V):
    """
    Perform a single Bellman backup.

    Parameters
    ----------
    state: int
    action: int
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    gamma: float
    V: np.array (num_states)

    Returns
    -------
    backup_val: float
    """
    backup_val = 0.
    
    ############################
    # YOUR IMPLEMENTATION HERE #
    backup_val = R[state, action] + gamma * np.sum(T[state, action] * V)
    ############################

    return backup_val

def policy_evaluation(policy, R, T, gamma, tol=1e-3):
    """
    Compute the value function induced by a given policy for the input MDP
    Parameters
    ----------
    policy: np.array (num_states)
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    gamma: float
    tol: float

    Returns
    -------
    value_function: np.array (num_states)
    """
    num_states, num_actions = R.shape
    value_function = np.zeros(num_states)
    
    ############################
    # YOUR IMPLEMENTATION HERE #
    while True:
        V = np.copy(value_function)
        for state in range(num_states):
            value_function[state] = bellman_backup(state, policy[state], R, T, gamma, V)
        if np.linalg.norm(V - value_function, ord=np.inf) < tol:
            break
    ############################

    return value_function


def policy_improvement(policy, R, T, V_policy, gamma):
    """
    Given the value function induced by a given policy, perform policy improvement
    Parameters
    ----------
    policy: np.array (num_states)
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)
    V_policy: np.array (num_states)
    gamma: float

    Returns
    -------
    new_policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    new_policy = np.zeros(num_states, dtype=int)

    ############################
    # YOUR IMPLEMENTATION HERE #
    new_policy = np.argmax(R + gamma * np.sum(T * V_policy, axis=2), axis=1)
    ############################
    
    return new_policy


def policy_iteration(R, T, gamma, tol=1e-3):
    """Runs policy iteration.

    You should call the policy_evaluation() and policy_improvement() methods to
    implement this method.
    Parameters
    ----------
    R: np.array (num_states, num_actions)
    T: np.array (num_states, num_actions, num_states)

    Returns
    -------
    V_policy: np.array (num_states)
    policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    V_policy = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)
    ############################
    # YOUR IMPLEMENTATION HERE #
    iter = 0
    while True:    
        iter += 1
        V_policy = policy_evaluation(policy, R, T, gamma, tol)
        new_policy = policy_improvement(policy, R, T, V_policy, gamma)

        if np.linalg.norm(policy - new_policy, ord=1) < tol:
            break
        else:
            policy = new_policy
    # print(f'Policy Iteration Rounds: {iter}')
    ############################
    
    return V_policy, policy


def value_iteration(R, T, gamma, tol=1e-3):
    """Runs value iteration.
    Parameters
    ----------
    R: np.array (num_states, num_actions)  b
    T: np.array (num_states, num_actions, num_states)

    Returns
    -------
    value_function: np.array (num_states)
    policy: np.array (num_states)
    """
    num_states, num_actions = R.shape
    value_function = np.zeros(num_states)
    policy = np.zeros(num_states, dtype=int)
    ############################
    # YOUR IMPLEMENTATION HERE #
    iter = 0
    while True:
        iter += 1
        V = np.copy(value_function)
        value_function = np.max(R + gamma * np.sum(T * V, axis=2), axis=1)
        if np.linalg.norm(value_function - V, ord=np.inf) < tol:
            break
    # print(f'Value Iternation Rounds: {iter}')

    policy = np.argmax(R + gamma * np.sum(T * value_function, axis=2), axis=1)
    ############################
    
    return value_function, policy


In [327]:
# Edit below to run policy and value iteration on different configurations
# You may change the parameters in the functions below
if __name__ == "__main__":
    SEED = 1234

    RIVER_CURRENT = 'WEAK' # 'WEAK' # 'MEDIUM'
    assert RIVER_CURRENT in ['WEAK', 'MEDIUM', 'STRONG']
    env = RiverSwim(RIVER_CURRENT, SEED)

    R, T = env.get_model()
    discount_factor = 0.5
    
    print("\n" + "-" * 25 + "\nBeginning Policy Iteration\n" + "-" * 25)

    V_pi, policy_pi = policy_iteration(R, T, gamma=discount_factor, tol=1e-3)
    print(V_pi)
    print([['L', 'R'][a] for a in policy_pi])

    print("\n" + "-" * 25 + "\nBeginning Value Iteration\n" + "-" * 25)

    V_vi, policy_vi = value_iteration(R, T, gamma=discount_factor, tol=1e-3)
    print(V_vi)
    print([['L', 'R'][a] for a in policy_vi])



-------------------------
Beginning Policy Iteration
-------------------------
[0.01  0.005 0.017 0.076 0.342 1.526]
['L', 'L', 'R', 'R', 'R', 'R']

-------------------------
Beginning Value Iteration
-------------------------
[0.01  0.005 0.017 0.076 0.342 1.526]
['L', 'L', 'R', 'R', 'R', 'R']


In [399]:
if __name__ == "__main__":
    SEED = 1234

    RIVER_CURRENT = 'WEAK' # 'WEAK' # 'MEDIUM'
    assert RIVER_CURRENT in ['WEAK', 'MEDIUM', 'STRONG']
    env = RiverSwim(RIVER_CURRENT, SEED)

    R, T = env.get_model()

    print("\n" + "-" * 135 + "\nSearching for the largest discount factor such that an optimal agent starting in the initial far-left state does not swim up the river\n" + "-" * 135)
    
    discount_factor = 1
    while True:
        discount_factor -= 0.01
        V_pi, policy_pi = policy_iteration(R, T, gamma=discount_factor, tol=1e-3)
        if np.all(policy_pi[0] == 0) | (discount_factor < 0):
            break
            
    print(f"River Current: {RIVER_CURRENT}")        
    print(f"Largest Discount Factor: {discount_factor: .2f}")
    print(f"State Value: {V_pi}")
    print(f"Policy: {[['L', 'R'][a] for a in policy_pi]}")


---------------------------------------------------------------------------------------------------------------------------------------
Searching for the largest discount factor such that an optimal agent starting in the initial far-left state does not swim up the river
---------------------------------------------------------------------------------------------------------------------------------------
River Current: WEAK
Largest Discount Factor:  0.67
State Value: [0.015 0.033 0.092 0.257 0.717 1.993]
Policy: ['L', 'R', 'R', 'R', 'R', 'R']


In [395]:
if __name__ == "__main__":
    SEED = 1234

    RIVER_CURRENT = 'MEDIUM' # 'WEAK' # 'MEDIUM'
    assert RIVER_CURRENT in ['WEAK', 'MEDIUM', 'STRONG']
    env = RiverSwim(RIVER_CURRENT, SEED)

    R, T = env.get_model()

    print("\n" + "-" * 135 + "\nSearching for the largest discount factor such that an optimal agent starting in the initial far-left state does not swim up the river\n" + "-" * 135)
    
    discount_factor = 1
    while True:
        discount_factor -= 0.01
        V_pi, policy_pi = policy_iteration(R, T, gamma=discount_factor, tol=1e-3)
        if np.all(policy_pi[0] == 0) | (discount_factor < 0):
            break
            
    print(f"River Current: {RIVER_CURRENT}")        
    print(f"Largest Discount Factor: {discount_factor: .2f}")
    print(f"State Value: {V_pi}")
    print(f"Policy: {[['L', 'R'][a] for a in policy_pi]}")


---------------------------------------------------------------------------------------------------------------------------------------
Searching for the largest discount factor such that an optimal agent starting in the initial far-left state does not swim up the river
---------------------------------------------------------------------------------------------------------------------------------------
River Current: MEDIUM
Largest Discount Factor:  0.77
State Value: [0.022 0.035 0.095 0.275 0.798 2.314]
Policy: ['L', 'R', 'R', 'R', 'R', 'R']


In [397]:
if __name__ == "__main__":
    SEED = 1234

    RIVER_CURRENT = 'STRONG' # 'WEAK' # 'MEDIUM'
    assert RIVER_CURRENT in ['WEAK', 'MEDIUM', 'STRONG']
    env = RiverSwim(RIVER_CURRENT, SEED)

    R, T = env.get_model()

    print("\n" + "-" * 135 + "\nSearching for the largest discount factor such that an optimal agent starting in the initial far-left state does not swim up the river\n" + "-" * 135)
    
    discount_factor = 1
    while True:
        discount_factor -= 0.01
        V_pi, policy_pi = policy_iteration(R, T, gamma=discount_factor, tol=1e-3)
        if np.all(policy_pi[0] == 0) | (discount_factor < 0):
            break
            
    print(f"River Current: {RIVER_CURRENT}")        
    print(f"Largest Discount Factor: {discount_factor: .2f}")
    print(f"State Value: {V_pi}")
    print(f"Policy: {[['L', 'R'][a] for a in policy_pi]}")


---------------------------------------------------------------------------------------------------------------------------------------
Searching for the largest discount factor such that an optimal agent starting in the initial far-left state does not swim up the river
---------------------------------------------------------------------------------------------------------------------------------------
River Current: STRONG
Largest Discount Factor:  0.93
State Value: [0.068 0.078 0.146 0.377 1.079 3.169]
Policy: ['L', 'R', 'R', 'R', 'R', 'R']
